## QFAMES with transverse field Ising model.

In [ ]:
import numpy as np
from scipy import linalg as LA
from scipy.signal import find_peaks

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsfonts, amssymb, amsmath, physics}')
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = ['serif']
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['font.size'] = 12



def plot_spectrum(gsEs, t, M_list, N_sample, L, g):
    # Compute G matrix around g.s.
    # Plot squared Frobenius norms of G matrix
    μ_vals = np.linspace(gsEs[0] - 0.3, gsEs[0] + 2, N_sample)
    norm2 = np.zeros_like(μ_vals, dtype=float)

    for idx, μ in enumerate(μ_vals):
        G = np.einsum('ijkl,ij->kl',M_list, np.exp(-np.subtract.outer(t, t) * 1j * μ))
        G /= N_sample ** 2
        norm2[idx] = np.linalg.norm(G, ord='fro')**2

    # Find peaks
    peaks, _ = find_peaks(norm2, height=0.5)
    
    # Plot
    plt.figure(figsize=(4.5,3.5))
    plt.plot(μ_vals, norm2, label=r'$\lVert G(\theta)\rVert_F^2$', lw=2)

    # Add vertical dashed lines indicating eigenvalues
    for idx, energy in enumerate(gsEs):
        plt.axvline(energy, color='gray', linestyle='--', linewidth=1,
                    label=r'$E_{%d} = %.2f$' % (idx + 1, energy))

    plt.xlabel(r'$\theta$')
    plt.title(r'$g = %.2f$' % (g))
    loc = 'upper right'
    bbox_to_anchor = None if g < 1.4 else (0.54, 1.02)
    plt.legend(frameon=False, loc=loc, bbox_to_anchor=bbox_to_anchor, fontsize=10, handlelength=1)
    plt.scatter(μ_vals[peaks[0]], norm2[peaks[0]], marker='*', color = 'red', s=50, zorder=100)
    plt.tight_layout()
    plt.savefig('figs/TFIM_spec_L%d_g%.2f.pdf' % (L, g), bbox_inches='tight')
    plt.close()

    # Return peaks of squared Frobenius norms
    return μ_vals[peaks]


def compute_and_plot_eigs(mu, t, M_list, N_sample, L, g, title='', thr=0.2):
    # Compute multiplicity at the peak of squared Frobenius norms (mu)
    # Threshold preset to be 0.05 -- should change w.r.t. noise
    # Plot the singular values of G matrix

    G = np.einsum('ijkl,ij->kl', M_list, np.exp(-np.subtract.outer(t, t) * 1j * mu))
    G /= (N_sample) ** 2

    plt.figure(figsize=(4, 3.5))
    eigvals = LA.eigvalsh(G)[::-1]


    x = np.arange(len(eigvals))
    plt.axhline(thr, color='red', linestyle='--', linewidth=2, label=r'$\tau$')
    plt.bar(x+1, eigvals, label=r'$\sigma(G(\theta^\star_1))$', color='blue', width=0.4)
    plt.xticks(x+1, fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=15)
    plt.tight_layout()
    plt.savefig('figs/TFIM_eigs_L%d_g%.2f.pdf' % (L, g),bbox_inches='tight')
    plt.close()

    # return multiplicity 
    return np.sum(np.where(eigvals > thr, 1, 0))


def compute_obs(mu, t, M_list, M_O_list, N_sample, multi):
    # Compute observable expectations at corresponding energy eigenstates    
    G = np.einsum('ijkl,ij->kl', M_list, np.exp(-np.subtract.outer(t, t) * 1j * mu))
    G /= (N_sample) ** 2

    GO = np.einsum('ijkl,ij->kl', M_O_list, np.exp(-np.subtract.outer(t, t) * 1j * mu))
    GO /= (N_sample) ** 2


    u, s, vh = LA.svd(G)

    G = (u[:,:multi]).conj().T @ G @ (vh.conj().T)[:,:multi]
    GO = (u[:,:multi]).conj().T @ GO @ (vh.conj().T)[:,:multi]

    eigvals = LA.eigvalsh(GO, b=G)  
    
    return eigvals


def plotData(fname, g, L, chi, N_sample, m, obs_names, noise_level = 0):
    t = np.loadtxt(f'{fname}_ts')
    gsEs = np.loadtxt(f'{fname}_gsEs')
    M_list = np.loadtxt(f'{fname}_M_list', dtype=np.complex128)
    M_list = M_list.reshape((N_sample, N_sample, m, m))

    
    tmax = 10
    ind_t = np.argwhere(np.abs(t) < tmax)[:, 0]
    t = t[ind_t]
    M_list = M_list[ind_t, :, :, :]
    M_list = M_list[:, ind_t, :, :]

    M_list += noise_level * np.random.randn(len(ind_t), len(ind_t), m, m)


    M_O_list = []
    for obs_idx, obs_name in enumerate(obs_names):
        l = np.loadtxt(f'{fname}_M_{obs_name}_list', dtype=np.complex128)
        l = l.reshape((N_sample, N_sample, m, m))
        l = l[ind_t, :, :, :]
        l = l[:, ind_t, :, :]
        l += noise_level * np.random.randn(len(ind_t), len(ind_t), m, m)
        M_O_list.append(l)

    peaks = plot_spectrum(gsEs, t, M_list, len(ind_t), L, g)

    μ = peaks[0]
    multi = compute_and_plot_eigs(μ, t, M_list, len(ind_t), L, g)
    print('g =', g, 'multi =', multi)
    
    vals = 0

    obs_val = []
    for obs_idx, obs_name in enumerate(obs_names):
        if (multi > 0):
            vals = compute_obs(μ, t, M_list, M_O_list[obs_idx], len(ind_t), multi)
        else:
            print('Error! No eigvals detected')
        if obs_name == 'Sx':
            vals = vals / L
        obs_val.append(vals)

    return peaks[0], multi, obs_val

In [ ]:
# System size
Ls = [20]

# Parameter for TFIM: g, and number of QFAMES initial states: m
gms = [[0.5, 5], [0.8, 5], [0.9, 5], [0.95, 5], [
    1.0, 5], [1.05, 5], [1.1, 5], [1.2, 5], [1.5, 5]]

# Observable names
obs_names_plot = [r'$S^z$']
obs_names = ['Sx']

N_sample = 100
epsilon = 0.2
chi = 100

colors = ['C1', 'C2', 'C3', 'C4', 'C5']
L = Ls[0]
print('L =', L)
peaks = []
multis = []
obs_vals = []
for g, m in gms:
    fname = f'data/tfi_chain_g{g}_L{L}_chi{chi}_N{N_sample}'
    peak, multi, obs_val = plotData(fname, g, L, chi, N_sample, m, obs_names)
    peaks.append(peak)
    multis.append(multi)
    obs_vals.append(obs_val)


for obs_idx, obs_name in enumerate(obs_names):
    plt.figure(figsize=(4, 3.5))
    for i in range(len(gms)):
        for j in range(multis[i]):
            plt.scatter(gms[i][0], obs_vals[i][obs_idx][j], edgecolor = 'C0', facecolor='C0')
            plt.vlines(gms[i][0], np.min(obs_vals[i][obs_idx]), np.max(obs_vals[i][obs_idx]), color = 'grey', lw = 2, zorder = -1)

    if obs_idx == 0:
        plt.axhline(0, color='grey', linestyle='--', linewidth=0.5)
    else:
        plt.axhline(1 / L, color='grey', linestyle='--', linewidth=0.5)
    plt.xlabel(r'$g$')
    plt.ylabel(obs_names_plot[obs_idx])
    plt.tight_layout()
    plt.savefig('figs/ob_%d_L%d.pdf' % (obs_idx, L),bbox_inches='tight')
    plt.close()